In [24]:
import tensorflow as tf
import PIL
from PIL.ImageDraw import Draw
import os
import numpy as np

In [14]:
IMAGE_SIZE = 299
LEARNING_RATE = 1e-3

In [2]:
def detect_objects(path: str, model) -> dict:
    """Function extracts image from a file, adds new axis
    and passes the image through object detection model.
    :param path: File path
    :param model: Object detection model
    :return: Model output dictionary
    """
    image_tensor = tf.image.decode_jpeg(
        tf.io.read_file(path), channels=3)[tf.newaxis, ...]
    return model(image_tensor)

In [3]:
def draw_bboxes(image_path, data: dict, threshold=0.) -> PIL.Image:
    """Function displays an image with bounding boxes
    overlaid for every detected person.
    :param image_path: File path to an image
    :param data: Output of objects detection model for this image
    :param threshold: Threshold for confidence scores
    :return: PIL.Image object
    """
    image = PIL.Image.open(image_path)
    draw = Draw(image)

    im_width, im_height = image.size

    boxes = data['detection_boxes'].numpy()[0]
    classes = data['detection_classes'].numpy()[0]
    scores = data['detection_scores'].numpy()[0]

    for i in range(int(data['num_detections'][0])):
        if classes[i] == 1 and scores[i] > threshold:
            ymin, xmin, ymax, xmax = boxes[i]
            (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                          ymin * im_height, ymax * im_height)
            draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)],
                      width=4, fill='red')

    return image

In [7]:
def create_model() -> tf.keras.Model:
  feature_model = tf.keras.applications.InceptionResNetV2(
  include_top=False, pooling='avg')
  feature_model.trainable = False

  model = tf.keras.Sequential([
      tf.keras.Input((IMAGE_SIZE, IMAGE_SIZE, 3)),
      feature_model,
      tf.keras.layers.Dense(512, activation='selu'),
      tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  return model

In [41]:
def get_image(path: str):
  image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
  image = tf.cast(image, dtype=tf.int32)
  image = tf.image.resize_with_pad(image, IMAGE_SIZE, IMAGE_SIZE)
  image = np.array(image, dtype=np.uint8)
  image = np.expand_dims(image, axis=0)

  return image

In [122]:
checkpoint_path = "drive/MyDrive/TIES4911/mini-project/training-2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'drive/MyDrive/TIES4911/mini-project/training-2/cp-0012.ckpt'

In [52]:
#latest = 'drive/MyDrive/TIES4911/mini-project/training/cp-0018.ckpt'

In [123]:
model = create_model()
model.load_weights(latest)

In [124]:
example_path = 'drive/MyDrive/TIES4911/mini-project/frames/seq_000002.jpg'
results = model.predict(get_image(example_path))
results

1/1 [==============================] - 4s 4s/step


array([[120.65214]], dtype=float32)

In [48]:
pred = results[0]
pr = pred.reshape((model.output_shape[1],  model.output_shape[2], model.output_shape[3])).argmax(axis=2)
print(np.max(pr))

IndexError: ignored

In [67]:
import scipy.io
mat = scipy.io.loadmat('drive/MyDrive/TIES4911/mini-project/mall_gt.mat')

In [60]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jul 09 10:58:59 2012',
 '__version__': '1.0',
 '__globals__': [],
 'frame': array([[array([[(array([[126.77986348,  60.70477816],
                        [116.95051195,  47.59897611],
                        [175.10750853,  44.3225256 ],
                        [184.11774744,  45.14163823],
                        [192.30887372,  44.3225256 ],
                        [230.80716724,  42.68430034],
                        [190.67064846,  66.43856655],
                        [176.74573379,  70.53412969],
                        [134.97098976, 100.0221843 ],
                        [ 14.56143345, 215.51706485],
                        [ 57.1552901 , 229.44197952],
                        [148.07679181, 193.40102389],
                        [143.16211604, 236.81399317],
                        [210.32935154, 222.8890785 ],
                        [260.29522184, 221.25085324],
                        [348.75938567, 225.

In [83]:
count = [[element for element in upperElement] for upperElement in mat['count']]
count[1]

[30]

In [85]:
frame = [[element for element in upperElement] for upperElement in mat['frame']]
len(frame[0][1999][0][0][0])

26

In [99]:
frame[0][1999][0][0][0]

array([[ 87.56365503, 283.86755647],
       [241.32135524, 228.6724846 ],
       [257.09137577, 243.12833676],
       [288.63141684, 252.98459959],
       [325.42813142, 258.89835729],
       [347.76899384, 229.98665298],
       [337.25564682, 338.40554415],
       [450.9312115 , 454.0523614 ],
       [421.362423  , 159.67864476],
       [454.87371663, 157.7073922 ],
       [515.9825462 , 170.84907598],
       [420.04825462,  84.11396304],
       [438.44661191,  79.51437372],
       [461.44455852,  47.31724846],
       [456.18788501,  44.03182752],
       [387.85112936,  43.37474333],
       [358.28234086,  93.31314168],
       [290.6026694 ,  97.91273101],
       [282.71765914, 105.14065708],
       [149.32956879,  99.22689938],
       [146.04414784,  82.79979466],
       [178.2412731 ,  49.94558522],
       [189.41170431,  45.34599589],
       [223.58008214,  38.775154  ],
       [223.58008214,  26.29055441],
       [263.00513347,  38.775154  ]])

In [ ]:
len(frame)

In [86]:
feat = scipy.io.loadmat('drive/MyDrive/TIES4911/mini-project/mall_feat.mat')
feat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Jul 18 14:34:08 2013',
 '__version__': '1.0',
 '__globals__': [],
 'x': array([[0.32839644, 0.29562692, 0.19459387, ..., 0.56384213, 0.59269831,
         0.5651216 ],
        [0.39574517, 0.32916678, 0.25529791, ..., 0.5407669 , 0.57611244,
         0.55281663],
        [0.42181511, 0.36246645, 0.2549268 , ..., 0.55810536, 0.56504656,
         0.55576596],
        ...,
        [0.09784577, 0.10210664, 0.0866383 , ..., 0.497792  , 0.5642227 ,
         0.49659307],
        [0.15068811, 0.15403319, 0.10343592, ..., 0.54465993, 0.58510344,
         0.54315362],
        [0.13216396, 0.13759431, 0.10544039, ..., 0.51964326, 0.57472731,
         0.5324382 ]])}

In [101]:
x = [[element for element in upperElement] for upperElement in feat['x']]
len(x[0])

30

In [94]:
tf.image.convert_image_dtype(x[0], np.uint8)

<tf.Tensor: shape=(30,), dtype=uint8, numpy=
array([ 83,  75,  49,  82,  79,  80,  78,  75, 128, 161, 102, 121, 148,
       135, 101, 111, 125, 158, 139, 139, 151, 142,  75,  73,  74,  73,
       141, 144, 151, 144], dtype=uint8)>

In [ ]:
import matplotlib.pyplot as plt

def display_image(image):
  fig = plt.figure(figsize=(10, 15))
  plt.grid(False)
  plt.imshow(image)
  plt.show()

display_image(tf.image.convert_image_dtype(x[0], np.uint8))

In [103]:
import pandas as pd

META_FILE = 'drive/MyDrive/TIES4911/mini-project/truth.csv'
data = pd.read_csv(META_FILE)


FileNotFoundError: ignored

In [116]:
61 % 60

1